## Reference - https://www.kaggle.com/yamqwe/great-barrier-reef-yolox-yolov5-ensemble

In [ ]:
import warnings; warnings.filterwarnings("ignore")

import os
import cv2
import ast
import sys
import glob
import torch
import shutil
import importlib
import traceback
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
sys.path.append('../input/tensorflow-great-barrier-reef')
tqdm.pandas()

from PIL import Image
from IPython.display import display
from tqdm.notebook import tqdm
tqdm.pandas()


### Install YoloX

In [ ]:
%cp -r /kaggle/input/yolox-cots-models /kaggle/working/
%cd /kaggle/working/yolox-cots-models/yolox-dep

In [ ]:
# Install YOLOX required modules
!pip install pip-21.3.1-py3-none-any.whl -f ./ --no-index
!pip install loguru-0.5.3-py3-none-any.whl -f ./ --no-index
!pip install ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl -f ./ --no-index
!pip install onnx-1.8.1-cp37-cp37m-manylinux2010_x86_64.whl -f ./ --no-index
!pip install onnxruntime-1.8.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl -f ./ --no-index
!pip install onnxoptimizer-0.2.6-cp37-cp37m-manylinux2014_x86_64.whl -f ./ --no-index
!pip install thop-0.0.31.post2005241907-py3-none-any.whl -f ./ --no-index
!pip install tabulate-0.8.9-py3-none-any.whl -f ./ --no-index

In [ ]:
# Install YOLOX
%cd /kaggle/working/yolox-cots-models/YOLOX
!pip install -r requirements.txt
!pip install -v -e . 

In [ ]:
# Install CocoAPI tool
%cd /kaggle/working/yolox-cots-models/yolox-dep/cocoapi/PythonAPI
!make
!make install
!python setup.py install

In [ ]:
import pycocotools

In [ ]:
# norfair dependencies
%cd /kaggle/input/norfair031py3/
!pip install commonmark-0.9.1-py2.py3-none-any.whl -f ./ --no-index
!pip install rich-9.13.0-py3-none-any.whl

!mkdir /kaggle/working/tmp
!cp -r /kaggle/input/norfair031py3/filterpy-1.4.5/filterpy-1.4.5/ /kaggle/working/tmp/
%cd /kaggle/working/tmp/filterpy-1.4.5/
!pip install .
!rm -rf /kaggle/working/tmp

# norfair
%cd /kaggle/input/norfair031py3/
!pip install norfair-0.3.1-py3-none-any.whl -f ./ --no-index

## YoloX

In [ ]:
%cd /kaggle/working/yolox-cots-models/YOLOX

# CHECKPOINT_FILE = '/kaggle/input/cotsyoloxse40ap2534stratifiedk/YOLOX_outputs/cots_config/best_ckpt.pth'
CHECKPOINT_FILE = '/kaggle/working/yolox-cots-models/yx_l_003.pth'

In [ ]:
config_file_template = '''

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 1
        self.width = 1
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]
        self.num_classes = 1

'''

with open('cots_config.py', 'w') as f:
    f.write(config_file_template)

In [ ]:
from yolox.utils import postprocess
from yolox.data.data_augment import ValTransform

COCO_CLASSES = (
  "starfish",
)

# get YOLOX experiment
current_exp = importlib.import_module('cots_config')
exp = current_exp.Exp()

# set inference parameters
test_size = (800, 1280)
num_classes = 1
confthre = 0.2
nmsthre = 0.2


# get YOLOX model
yolox_model = exp.get_model()
yolox_model.cuda()
yolox_model.eval()

# get custom trained checkpoint
ckpt_file = CHECKPOINT_FILE
ckpt = torch.load(ckpt_file, map_location="cpu")
yolox_model.load_state_dict(ckpt["model"])

In [ ]:
def yolox_inference(img, model, test_size): 
    bboxes = []
    bbclasses = []
    scores = []
    
    preproc = ValTransform(legacy = False)

    tensor_img, _ = preproc(img, None, test_size)
    tensor_img = torch.from_numpy(tensor_img).unsqueeze(0)
    tensor_img = tensor_img.float()
    tensor_img = tensor_img.cuda()

    with torch.no_grad():
        outputs = model(tensor_img)
        outputs = postprocess(
                    outputs, num_classes, confthre,
                    nmsthre, class_agnostic=True
                )

    if outputs[0] is None:
        return [], [], []
    
    outputs = outputs[0].cpu()
    bboxes = outputs[:, 0:4]

    bboxes /= min(test_size[0] / img.shape[0], test_size[1] / img.shape[1])
    bbclasses = outputs[:, 6]
    scores = outputs[:, 4] * outputs[:, 5]
    
    return bboxes, bbclasses, scores

In [ ]:
def draw_yolox_predictions(img, bboxes, scores, bbclasses, confthre, classes_dict):
    for i in range(len(bboxes)):
            box = bboxes[i]
            cls_id = int(bbclasses[i])
            score = scores[i]
            if score < confthre:
                continue
            x0 = int(box[0])
            y0 = int(box[1])
            x1 = int(box[2])
            y1 = int(box[3])

            cv2.rectangle(img, (x0, y0), (x1, y1), (0, 255, 0), 2)
            cv2.putText(img, '{}:{:.1f}%'.format(classes_dict[cls_id], score * 100), (x0, y0 - 3), cv2.FONT_HERSHEY_PLAIN, 0.8, (0,255,0), thickness = 1)
    return img

## YoloV5

In [ ]:
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
CKPT_PATH = '/kaggle/input/yolov5m6/best(1).pt'
IMG_SIZE  = 6000
CONF      = 0.01
IOU       = 0.4
AUGMENT   = True

In [ ]:
def get_path(row):
    row['image_path'] = f'{ROOT_DIR}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

In [ ]:
# Train Data
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df = df.progress_apply(get_path, axis=1)
df['annotations'] = df['annotations'].progress_apply(lambda x: ast.literal_eval(x))
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts()/len(df)*100

In [ ]:
def voc2yolo(bboxes, image_height=720, image_width=1280):
    """
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    return bboxes

def yolo2voc(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    return bboxes

def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    return bboxes

def yolo2coco(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    """
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    # converstion (xmid, ymid) => (xmin, ymin)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    return bboxes

def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes  = voc2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2coco(bboxes, image_height, image_width)
    return bboxes

def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def draw_bboxes(img, bboxes, classes, class_ids, colors = None, show_classes = None, bbox_format = 'yolo', class_name = False, line_thickness = 2):

    image = img.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255 ,0) if colors is None else colors

    if bbox_format == 'yolo':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:

                x1 = round(float(bbox[0])*image.shape[1])
                y1 = round(float(bbox[1])*image.shape[0])
                w  = round(float(bbox[2])*image.shape[1]/2) #w/2
                h  = round(float(bbox[3])*image.shape[0]/2)

                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(get_label(cls)),
                             line_thickness = line_thickness)

    elif bbox_format == 'coco':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                w  = int(round(bbox[2]))
                h  = int(round(bbox[3]))

                voc_bbox = (x1, y1, x1+w, y1+h)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)

    elif bbox_format == 'voc_pascal':

        for idx in range(len(bboxes)):

            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors

            if cls in show_classes:
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                voc_bbox = (x1, y1, x2, y2)
                plot_one_box(voc_bbox,
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)
    else:
        raise ValueError('wrong bbox format')

    return image

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

In [ ]:
##############################################################
#                      Tracking helpers                      #
##############################################################

import numpy as np
from norfair import Detection, Tracker

# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result

# Euclidean distance function to match detections on this frame with tracked_objects from previous frames
def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
def load_model(ckpt_path, conf=0.15, iou=0.20):
    model = torch.hub.load('/kaggle/input/yolov5-lib-ds',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
#     model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
    return model

In [ ]:
def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    if len(bboxes):
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        confs   = preds.confidence.values
        return bboxes, confs
    else:
        return [],[]

def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

def show_img(img, bboxes, bbox_format='yolo', bbox_colors = None):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes,
                           classes = names,
                           class_ids = labels,
                           class_name = True,
                           colors = colors if bbox_colors is None else bbox_colors,
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((800, 400))

In [ ]:
def tracking_function(tracker, frame_id, bboxes, scores):
    
    detects = []
    predictions = []
    
    if len(scores)>0:
        for i in range(len(bboxes)):
            box = bboxes[i]
            score = scores[i]
            x_min = int(box[0])
            y_min = int(box[1])
            bbox_width = int(box[2])
            bbox_height = int(box[3])
            if score > 0.15:
                detects.append([x_min, y_min, x_min+bbox_width, y_min+bbox_height, score])
                predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
#             print(predictions[:-1])
    # Update tracks using detects from current frame
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
        if last_detected_frame_id + 1 != frame_id:  # Track only one frame forward
            continue
        # Add objects that have no detections on current frame to predictions
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
        score = tobj.last_detection.scores[0]

        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
        
    return predictions



In [ ]:
def CLAHE(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized = clahe.apply(gray)
    return equalized
def Gamma_enhancement(image):
    gamma = 1/0.6
    R = 255.0
    return (R * np.power(image.astype(np.uint32)/R, gamma)).astype(np.uint8)

## Ensembling + Tracking on Train

In [ ]:
import sys; sys.path.append('/kaggle/input/weightedboxesfusion/')

In [ ]:
model = load_model(CKPT_PATH, conf=CONF, iou=IOU)

In [ ]:
def run_wbf(bboxes, confs, image_size=512, iou_thr=0.2, skip_box_thr=0.001, weights=None):
    boxes =  [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

from ensemble_boxes import *

In [ ]:
# TEST_IMAGE_PATH = "/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/9674.jpg"

# from ensemble_boxes import *

# def run_wbf(bboxes, confs, image_size=512, iou_thr=0.2, skip_box_thr=0.001, weights=None):
#     boxes =  [bbox/(image_size-1) for bbox in bboxes]
#     scores = [conf for conf in confs]    
#     labels = [np.ones(conf.shape[0]) for conf in confs]
#     boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
#     boxes = boxes*(image_size-1)
#     return boxes, scores, labels

# model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
# image_paths = df[df.num_bbox>1].sample(3).image_path.tolist()

# tracker = Tracker(
#     distance_function=euclidean_distance, 
#     distance_threshold=30,
#     hit_inertia_min=3,
#     hit_inertia_max=6,
#     initialization_delay=1,
# )

# frame_id =0
# for idx, path in enumerate(image_paths):
#     img = cv2.imread(path)[...,::-1]
    
#     bboxes_1, bbclasses, scores = yolox_inference(img[...,::-1], yolox_model, test_size)        
#     bboxes_2, confis = predict(model, img, size=IMG_SIZE, augment=AUGMENT)        
#     pred_1, pred_2 = voc2coco(bboxes_1.detach().numpy(),img.shape[1],img.shape[2]).astype(int), bboxes_2
#     print(pred_1)
#     print(pred_2)
#     print(scores)
#     print(confis)
#     if len(pred_1) > 0 and len(pred_2) > 0: boxes, scores, labels = run_wbf([pred_1, pred_2], [scores, confis], image_size = IMG_SIZE)
#     elif len(pred_1) > 0: boxes, scores = bboxes_1.detach().numpy(), scores
#     elif len(pred_2) > 0: boxes, scores = bboxes_2.detach().numpy(), confis
        
#     predictions = tracking_function(tracker, frame_id, boxes, scores)
    
#     if frame_id < 7:
#         if len(predictions)>0:
#             box = [list(map(int,box.split(' ')[1:])) for box in predictions]
#         else:
#             box = []
#         print('\n Tracking')
#         display(show_img(img, box, bbox_format='coco'))

#     frame_id += 1   
#     print('\nEnsemble (WBF) Predictions: ')
#     display(show_img(img, boxes, bbox_format='coco'))
#     print("******************************************************************")
    
#     if idx>5:
#         break

In [ ]:
tracker1 = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)
tracker2 = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)
tracker3 = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)

In [ ]:
frame_id_1 = 0
frame_id_2 = 0 
frame_id_3 = 0

image_paths = df[df.num_bbox>1].sample(100).image_path.tolist()
for idx, path in enumerate(image_paths):
    img = cv2.imread(path)[...,::-1]
    conf1 = []
    conf2 = []
    tmp1 = []
    tmp2 = []
    bboxes_1, bbclasses, scores = yolox_inference(img[...,::-1], yolox_model, test_size)  
    bboxes_1 = voc2coco(bboxes_1.detach().numpy(),img.shape[1],img.shape[2]).astype(int) #change to coco
#     scores = scores.detach().numpy().reshape((1,len(scores)))[0]
    
    bboxes_2, confis = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
#     print(bboxes_1)
    
    pred_1 =  tracking_function(tracker1, frame_id_1, bboxes_1, scores)
    pred_2 =  tracking_function(tracker2, frame_id_2, bboxes_2, confis)
#     print(pred_1)
#     print(pred_2)
    
    box1 = [list(map(int,box.split(' ')[1:])) for box in pred_1]
    box2 = [list(map(int,box.split(' ')[1:])) for box in pred_2]

    
    for box in pred_1:
        conf1.append(float(box.split(' ')[0]))
    conf1 = np.asarray(conf1)
    for box in pred_2:
        conf2.append(float(box.split(' ')[0]))
    conf2 = np.asarray(conf2)
#     print(conf1)
#     print(conf2)

    for i in box1:
        a = []
        for j in i :
            a.append(j)
        c = np.asarray(a)
        tmp1.append(c)
    box1 = np.asarray(tmp1)
    
    for i in box2:
        a = []
        for j in i :
            a.append(j)
        c = np.asarray(a)
        tmp2.append(c)
    box2 = np.asarray(tmp2)
#     print(box1)
#     print(box2)

    boxes, scores, labels = run_wbf([box1,box2], [conf1,conf2], image_size = 512)
#     print(boxes)
    predictions = tracking_function(tracker3, frame_id_3, boxes, scores)
#     print(predictions)
    
    print('\nEnsemble (WBF) Predictions: ')
    display(show_img(img, boxes, bbox_format='coco'))
    
    if frame_id_1 < 7:
        if len(predictions)>0:
            box = [list(map(int,box.split(' ')[1:])) for box in predictions]
        else:
            box = []
        print('\n Tracking')
        display(show_img(img, box, bbox_format='coco'))
    print('--------------------------------------')
    frame_id_1 += 1   
    frame_id_2 += 1
    frame_id_3 += 1
    
    if idx>5:
        break

## Submit Prediction

In [ ]:
%cd /kaggle/working/
import greatbarrierreef

env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()  

In [ ]:
tracker1 = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)
tracker2 = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)
tracker3 = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)

In [ ]:
submission_dict = {
    'id': [],
    'prediction_string': [],
}
frame_id_1 = 0 
frame_id_2 = 0 
frame_id_3 = 0
for (image_np, pred_df) in iter_test:
    
    conf1 = []
    conf2 = []
    tmp1 = []
    tmp2 = []   
    
    bboxes_1, bbclasses, scores = yolox_inference(image_np[...,::-1], yolox_model, test_size)
    bboxes_2, confis = predict(model, image_np, size=IMG_SIZE, augment=AUGMENT)  
    print(bboxes_1)
    print(bboxes_2)
    if len(bboxes_1) > 0 and len(bboxes_2) > 0: 
        bboxes_1, bboxes_2 = voc2coco(bboxes_1.detach().numpy(), image_np.shape[1],image_np.shape[2]).astype(int), bboxes_2
        pred_1 =  tracking_function(tracker1, frame_id_1, bboxes_1, scores)
        pred_2 =  tracking_function(tracker2, frame_id_2, bboxes_2, confis)

        box1 = [list(map(int,box.split(' ')[1:])) for box in pred_1]
        box2 = [list(map(int,box.split(' ')[1:])) for box in pred_2]


        for box in pred_1:
            conf1.append(float(box.split(' ')[0]))
        conf1 = np.asarray(conf1)
        for box in pred_2:
            conf2.append(float(box.split(' ')[0]))
        conf2 = np.asarray(conf2)
    #     print(conf1)
    #     print(conf2)

        for i in box1:
            a = []
            for j in i :
                a.append(j)
            c = np.asarray(a)
            tmp1.append(c)
        box1 = np.asarray(tmp1)

        for i in box2:
            a = []
            for j in i :
                a.append(j)
            c = np.asarray(a)
            tmp2.append(c)
        box2 = np.asarray(tmp2)
    #     print(box1)
    #     print(box2)

        bboxes, scores, labels = run_wbf([box1,box2], [conf1,conf2], image_size = IMG_SIZE)
#         bboxes, scores, labels = run_wbf([pred_1, pred_2], [scores, confis], image_size = IMG_SIZE)
    elif len(bboxes_1) > 0: bboxes, scores = voc2coco(bboxes_1.detach().numpy(), image_np.shape[1], image_np.shape[2]).astype(int), scores        
    elif len(bboxes_2) > 0: bboxes, scores = bboxes_2, confis        
    else: bboxes = []

    predictions = tracking_function(tracker3, frame_id_3, bboxes, scores)
#     predictions = tracking_function(tracker, frame_id, bboxes, scores)
    
    prediction_str = ' '.join(predictions)
    pred_df['annotations'] = prediction_str
    env.predict(pred_df)
    if frame_id_1 < 3:
        if len(predictions)>0:
            box = [list(map(int,box.split(' ')[1:])) for box in predictions]
        else:
            box = []
        display(show_img(image_np, box, bbox_format='coco'))
#     print('Prediction:', pred_df)
    frame_id_1 += 1
    frame_id_2 += 1
    frame_id_3 += 1

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()